### Transfer Learning

In this part, we shall use pretrained network VGG-16 to extract image features from the cats and dog images. Then we will use a simple Multilayer perceptron to classify the images using the above extracted features as inputs.

### Data loading

In [33]:
##This notebook is built around using tensorflow as the backend for keras
!pip install pillow
!KERAS_BACKEND=tensorflow python -c "from keras import backend"

/bin/sh: 1: python: not found


In [76]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
import scipy
import pylab as pl
import matplotlib.cm as cm
%matplotlib inlineimport keras
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator,array_to_img, img_to_array,load_img
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
from matplotlib import pyplot as plt

UsageError: unrecognized arguments: keras


In [35]:
# dimensions of our images.
img_width, img_height = 50, 50

train_data_dir = 'data/train/'
validation_data_dir = 'data/validation/'

## Using a pre-trained model

The process of training a convolutionnal neural network can be very time-consuming and require a lot of datas.  

We can go beyond the previous models in terms of performance and efficiency by using a general-purpose, pre-trained image classifier.  This example uses VGG16, a model trained on the ImageNet dataset - which contains millions of images classified in 1000 categories. 

On top of it, we add a small multi-layer perceptron and we train it on our dataset.

## VGG-16 Network

The VGG-16 is a 16-layer network used by the VGG team in the ILSVRC-2014 competition. The network architecture and implementation details can be found in this [paper](https://arxiv.org/abs/1409.1556)


## Transfer Learning

Transfer learning allows us to deal with these scenarios by leveraging the already existing labeled data of some related task or domain. We try to store this knowledge gained in solving the source task in the source domain and apply it to our problem of interest. This knowledge can take on various forms depending on the data: it can pertain to how objects are composed to allow us to more easily identify novel objects; it can be with regard to the general words people use to express their opinions, etc.

#### VGG16 model architecture definition

In [36]:
model_vgg = Sequential()
model_vgg.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
model_vgg.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", name="conv1_1")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", name="conv1_2")`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", name="conv2_1")`
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", name="conv2_2")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to

#### Loading VGG16 weights

*Note : the VGG16 weights file (~500MB) is not included in this repository. You can download from here :  
https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3*

In [37]:
import h5py
f = h5py.File('models/vgg/vgg16_weights.h5')
for k in range(f.attrs['nb_layers']):
    if k >= len(model_vgg.layers) - 1:
        # we don't look at the last two layers in the savefile (fully-connected and activation)
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    layer = model_vgg.layers[k]

    #if layer.__class__.__name__ in ['Convolution1D', 'Convolution2D', 'Convolution3D', 'AtrousConvolution2D']:
    if weights:
        weights[0] = np.transpose(weights[0], (2, 3, 1, 0))
        #print(weights[0].shape)
    layer.set_weights(weights)

f.close()

### Using the VGG16 model to process samples

In [38]:
imageDataGenerator = ImageDataGenerator()
train_generator_bottleneck = imageDataGenerator.flow_from_directory(train_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode=None,
        shuffle=False)

validation_generator_bottleneck = imageDataGenerator.flow_from_directory(validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode=None,
        shuffle=False)

Found 1402 images belonging to 2 classes.
Found 101 images belonging to 2 classes.


This is a long process, so we save the output of the VGG16 once and for all.  

In [39]:
bottleneck_features_train = model_vgg.predict_generator(train_generator_bottleneck, 1400)
np.save(open('models/bottleneck_features_train.npy', 'wb'), bottleneck_features_train)

In [40]:
bottleneck_features_validation = model_vgg.predict_generator(validation_generator_bottleneck, 100)
np.save(open('models/bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)

Now we can load it...

In [41]:
train_data = np.load(open('models/bottleneck_features_train.npy', 'rb'))
train_labels = np.array([0] * (1400 // 2) + [1] * (1400 // 2))

validation_data = np.load(open('models/bottleneck_features_validation.npy', 'rb'))
validation_labels = np.array([0] * (100 // 2) + [1] * (100 // 2))

And define and train the custom fully connected neural network :

In [42]:
model_top = Sequential()
model_top.add(Flatten(input_shape=train_data.shape[1:]))
model_top.add(Dense(256, activation='relu'))
model_top.add(Dropout(0.5))
model_top.add(Dense(1, activation='sigmoid'))

model_top.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [43]:
epochs=40
model_top.fit(train_data[:1400], train_labels,
          epochs=epochs, batch_size=32,
          validation_data=(validation_data[:100], validation_labels))

Train on 1400 samples, validate on 100 samples
Epoch 1/40
1400/1400 [==============================] - 3s 2ms/step - loss: 7.9834 - acc: 0.5036 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 2/40
1400/1400 [==============================] - 1s 811us/step - loss: 6.6480 - acc: 0.5786 - val_loss: 6.9835 - val_acc: 0.5600
Epoch 3/40
1400/1400 [==============================] - 1s 756us/step - loss: 3.9570 - acc: 0.7443 - val_loss: 3.1671 - val_acc: 0.7700
Epoch 4/40
1400/1400 [==============================] - 1s 762us/step - loss: 3.3398 - acc: 0.7836 - val_loss: 3.6541 - val_acc: 0.7700
Epoch 5/40
1400/1400 [==============================] - 1s 799us/step - loss: 2.6386 - acc: 0.8257 - val_loss: 4.4832 - val_acc: 0.7200
Epoch 6/40
1400/1400 [==============================] - 1s 809us/step - loss: 2.7758 - acc: 0.8186 - val_loss: 2.8786 - val_acc: 0.8100
Epoch 7/40
1400/1400 [==============================] - 1s 781us/step - loss: 2.5673 - acc: 0.8307 - val_loss: 2.9608 - val_acc: 0.8100
Epo

The training process of this small neural network is very fast : ~37s per epoch

In [44]:
model_top.save_weights('models/bottleneck_40_epochs.h5')

### Bottleneck model evaluation

In [45]:
model_top.load_weights('models/bottleneck_40_epochs.h5')
#model_top.load_weights('/notebook/Data1/Code/keras-workshop/models/with-bottleneck/1000-samples--100-epochs.h5')

In [67]:
model_top.evaluate(validation_data[:100], validation_labels)

100/100 [==============================] - 0s 341us/step


[2.5595671844482424, 0.84]

Loss and accuracy :

**We reached a 84% accuracy on the validation after 40 epochs, which is a massive boost on our baseline model!**

Next, we shall try and finetune the model so that it adapts on this dataset, in order to further boost the accuracy.